In [38]:
import xmltodict
import cv2
import os
import zipfile
import shutil
import sys
import random
import csv

In [9]:
def extract_directory(path_to_zip_file):
    """Extract the zip file on the given path to a directory under the same name."""
    path_to_zip_file_list = path_to_zip_file.split('/')
    path_to_zip_len = len(path_to_zip_file_list)
    path_to_extracted_file = '/' + os.path.join(*path_to_zip_file_list[:path_to_zip_len - 1])
    destination_dir_name = path_to_zip_file_list[path_to_zip_len - 1].split('.')[0]
    zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
    zip_ref.extractall(path_to_extracted_file)
    zip_ref.close()
    # Remove an unnecessary automatically created directory.
    shutil.rmtree(os.path.join(path_to_extracted_file, "__MACOSX"))
    return os.path.join(path_to_extracted_file, destination_dir_name)

PATH_TO_ZIP_FILE = "/Users/alejandrosanchezaristizabal/Desktop/annotations_2.zip"
annotations_path = extract_directory(PATH_TO_ZIP_FILE)
print(annotations_path)

/Users/alejandrosanchezaristizabal/Desktop/annotations_2


In [10]:
PATH_TO_IMG_ZIP_FILE = "/Users/alejandrosanchezaristizabal/Desktop/images_2.zip"
images_path = extract_directory(PATH_TO_IMG_ZIP_FILE)
print(images_path)

/Users/alejandrosanchezaristizabal/Desktop/images_2


In [11]:
def get_annotation_file_paths(annotation_path):
    """Get a list with the XML file paths."""
    xml_file_paths = list()
    for subdir, dirs, files in os.walk(annotations_path):
        for file in files:
            file_path = os.path.join(subdir, file)
            if file_path.endswith(".xml"):
                xml_file_paths.append(file_path)
    return xml_file_paths

xml_file_paths = get_annotation_file_paths(ANNOTATIONS_PATH)
print(xml_file_paths)

['/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/12.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/13.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/10.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/4.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/5.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/7.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/2.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/3.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/1.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/0.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/4/14.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/4/15.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/4/17.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/4/16.xml', '/Users/alejandrosanchezaristizabal/Desk

In [12]:
def get_info_from_object_item(object_item, info):
    """Updates the given dictionary using the values of the given object item."""
    label = object_item["name"]
    bnd_box = object_item["bndbox"]
    x_min = int(bnd_box["xmin"])
    y_min = int(bnd_box["ymin"])
    x_max = int(bnd_box["xmax"])
    y_max = int(bnd_box["ymax"])
    info["label"] = label
    info["xmin"] = x_min
    info["ymin"] = y_min
    info["xmax"] = x_max
    info["ymax"] = y_max
    return info

def parse_xml(document):
    """Parse the given document in order to extract the relevant values."""
    folder = document["annotation"]["folder"]
    filename = document["annotation"]["filename"]
    infos = list()
    object_items = document["annotation"]["object"]
    if isinstance(object_items, list):
        # The file contains more than one bounding-box object.
        for object_item in object_items:
            info = dict()
            info["folder"] = folder
            info["filename"] = filename
            info = get_info_from_object_item(object_item, info)
            infos.append(info)
    else:
        info = dict()
        info["folder"] = folder
        info["filename"] = filename
        info = get_info_from_object_item(object_items, info)
        infos.append(info)
    return infos

def get_info_from_xml_files(xml_file_paths):
    """Get the labelling information within each XML file."""
    info_from_xml_files = list()
    for xml_file_path in xml_file_paths:
        with open(xml_file_path) as fd:
            document = xmltodict.parse(fd.read())
            for info in parse_xml(document):
                info_from_xml_files.append(info)
    return info_from_xml_files

XML_FILE_PATHS = xml_file_paths
info_from_xml_files = get_info_from_xml_files(XML_FILE_PATHS)

In [59]:
def update_metadata(file_path, img_path, label, belonging_set):
    """Update the metadata.csv file with the given information."""
    columns = ["img_path", "label", "set"]
    if not os.path.isfile(file_path):
        with open(file_path, 'w') as metadata_csv:
            metadata_writer = csv.DictWriter(metadata_csv, columns)
            metadata_writer.writeheader()
            row = {columns[0]: img_path, columns[1]: label, columns[2]: belonging_set}
            metadata_writer.writerow(row)
    else:
        with open(file_path, 'a') as metadata_csv:
            metadata_writer = csv.DictWriter(metadata_csv, columns)
            row = {columns[0]: img_path, columns[1]: label, columns[2]: belonging_set}
            metadata_writer.writerow(row)

def belongs_to(train_threshold):
    """Decide to which of the three sets (train, valid, test) an element belongs."""
    train_threshold_percentage = train_threshold / 100
    belonging_set = None
    random_num = random.random()
    if random_num <= train_threshold_percentage:
        belonging_set = "train"
    elif train_threshold_percentage < random_num <= train_threshold_percentage +  \
        (1 - train_threshold_percentage) / 2:
        belonging_set = "valid"
    else:
        belonging_set = "test"
    return belonging_set

def get_unique_name(img_path):
    """Create a unique name for the image on the given path."""
    fn = lambda x: str(hash(x) % ((sys.maxsize + 1) * 2)) + ".JPG"
    return fn(img_path)

def crop_img(img, info_from_xml_file):
    """Crop the image according to the coordinates in the info_from_xml_file."""
    x_min = info_from_xml_file["xmin"]
    y_min = info_from_xml_file["ymin"]
    x_max = info_from_xml_file["xmax"]
    y_max = info_from_xml_file["ymax"]
    return img[y_min:y_max, x_min:x_max, :]

def create_metadata(images_path, info_from_xml_files, train_percentage, no_annotation_dir):
    """Create the metadata.csv file using the images and the information extracted as XML files."""
    cropped_img_directory = "{}_cropped".format(images_path)
    images_path_as_list = images_path.split('/')
    metadata_file_path = '/'.join(images_path_as_list[:len(images_path_as_list) - 1]) +  \
        "/metadata.csv"
    if os.path.exists(cropped_img_directory):
        shutil.rmtree(cropped_img_directory)
    os.makedirs(cropped_img_directory)
    if os.path.exists(metadata_file_path):
        os.remove(metadata_file_path)
    for info_from_xml_file in info_from_xml_files:
        img_path_from_xml = "{}/{}/{}".format(images_path, info_from_xml_file["folder"], 
                                              info_from_xml_file["filename"])
        current_img = cv2.imread(img_path_from_xml)
        cropped_img = crop_img(current_img, info_from_xml_file)
        cropped_img_name = get_unique_name(img_path_from_xml)
        img_path_destination = "{}/{}".format(cropped_img_directory, cropped_img_name)
        belonging_set = belongs_to(train_percentage)
        cv2.imwrite(img_path_destination, cropped_img)
        metadata_path = update_metadata(metadata_file_path, img_path_destination, 
                                        info_from_xml_file["label"], belonging_set)
    
    # Add the data of the "no_annotation_directory" to the created metadata.csv file.
    for subdir, dirs, files in os.walk("{}/{}".format(images_path, no_annotation_dir)):
        for file in files:
            file_path = os.path.join(subdir, file)
            if not file_path.endswith(".DS_Store"):
                current_img = cv2.imread(file_path)
                img_name = get_unique_name(file_path)
                img_path_destination = "{}/{}".format(cropped_img_directory, img_name)
                belonging_set = belongs_to(train_percentage)
                cv2.imwrite(img_path_destination, current_img)
                metadata_path = update_metadata(metadata_file_path, img_path_destination, '0', 
                                                belonging_set)
    print("The metadata.csv file was successfully generated.")
    return metadata_path

IMAGES_PATH = images_path
INFO_FROM_XML_FILES = info_from_xml_files
TRAIN_PERCENTAGE = 70
NO_ANNOTATION_DIR = "0 - No hay persona"
metadata_path = create_metadata(IMAGES_PATH, INFO_FROM_XML_FILES, TRAIN_PERCENTAGE, 
                                NO_ANNOTATION_DIR)

The metadata.csv file was successfully generated.
